# Business Problem
In Manhattan, we have a business problem. It is to open a new restaurant.

1. We will use the neighbourhood data in New York.
2. After K-means clustering, 1 cluster will be chosen.
3. Best place for new restaurant will be finally selected.

# Describing Data 

Manhattan data of New York will be used for new restaurant position analysis. Foursquare location data will be used for solving the problem.

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [2]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [3]:
neighborhoods_data = newyork_data['features']

In [4]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [5]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [6]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [7]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [8]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [24]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [10]:
CLIENT_ID = 'BCCC3IQA4YDRPD3KSSDQGKWUAKFA4KZOZCOCO5ULOULVNT03' # your Foursquare ID
CLIENT_SECRET = 'FDCGUG2FKT3RMKMPEQQZEDFYFTZRNK5VPEONKCLPS0SPQRXA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BCCC3IQA4YDRPD3KSSDQGKWUAKFA4KZOZCOCO5ULOULVNT03
CLIENT_SECRET:FDCGUG2FKT3RMKMPEQQZEDFYFTZRNK5VPEONKCLPS0SPQRXA


In [11]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=BCCC3IQA4YDRPD3KSSDQGKWUAKFA4KZOZCOCO5ULOULVNT03&client_secret=FDCGUG2FKT3RMKMPEQQZEDFYFTZRNK5VPEONKCLPS0SPQRXA&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=50'

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c5bf3901ed2193b433f07e4'},
 'response': {'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 24,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981574,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.87441177110231,
          'lng': -73.91027100981574}],
        'distance'

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Land & Sea Restaurant,Seafood Restaurant,40.877885,-73.905873


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

24 venues were returned by Foursquare.


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )



Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [18]:
print(manhattan_venues.shape)
manhattan_venues.head()

(1892, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant


In [19]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Trail,Tree,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Watch Shop,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's S

In [20]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Trail,Tree,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Watch Shop,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's S

In [21]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
          venue  freq
0          Park  0.08
1   Coffee Shop  0.08
2      Fountain  0.04
3     BBQ Joint  0.04
4  Cupcake Shop  0.04


----Carnegie Hill----
                venue  freq
0                 Spa  0.06
1         Pizza Place  0.06
2           Bookstore  0.06
3  Italian Restaurant  0.04
4              Bakery  0.04


----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1   Chinese Restaurant  0.05
2  American Restaurant  0.05
3       Cosmetics Shop  0.05
4    French Restaurant  0.05


----Chelsea----
                venue  freq
0      Ice Cream Shop  0.08
1               Hotel  0.06
2           Nightclub  0.06
3    Asian Restaurant  0.04
4  Seafood Restaurant  0.04


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.12
1      Sandwich Place  0.04
2        Noodle House  0.04
3    Asian Restaurant  0.04
4      Ice Cream Shop  0.04


----Civic Center----
                  venue  freq
0                Bakery 

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Plaza,BBQ Joint,Fountain,Sandwich Place,Food Court,Cupcake Shop,Department Store,Pizza Place
1,Carnegie Hill,Spa,Bookstore,Pizza Place,Yoga Studio,Italian Restaurant,French Restaurant,Coffee Shop,Café,Gym / Fitness Center,Bakery
2,Central Harlem,African Restaurant,Seafood Restaurant,Fried Chicken Joint,American Restaurant,Chinese Restaurant,French Restaurant,Gym / Fitness Center,Cosmetics Shop,Bookstore,Library
3,Chelsea,Ice Cream Shop,Hotel,Nightclub,Seafood Restaurant,Theater,Bakery,Coffee Shop,Asian Restaurant,Italian Restaurant,Speakeasy
4,Chinatown,Chinese Restaurant,Asian Restaurant,Sandwich Place,American Restaurant,Ice Cream Shop,Bar,Noodle House,Vietnamese Restaurant,Salon / Barbershop,Malay Restaurant


# Methodology

1. K-means clustering method will be applied for selecting narrow-down area. 
2. In selected area, low frequency of restaurants will be selected for the position of new restaurant open.

In [25]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 2, 2, 2, 1, 4, 4, 0])

In [26]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Discount Store,Coffee Shop,Yoga Studio,Pizza Place,Supplement Shop,Steakhouse,Shopping Mall,Shoe Store,Seafood Restaurant,Sandwich Place
1,Manhattan,Chinatown,40.715618,-73.994279,2,Chinese Restaurant,Asian Restaurant,Sandwich Place,American Restaurant,Ice Cream Shop,Bar,Noodle House,Vietnamese Restaurant,Salon / Barbershop,Malay Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Café,Caribbean Restaurant,Chinese Restaurant,Bakery,Mobile Phone Shop,Latin American Restaurant,Tapas Restaurant,Park,Deli / Bodega,Wine Shop
3,Manhattan,Inwood,40.867684,-73.921210,4,Mexican Restaurant,Café,Pizza Place,Bakery,Park,Restaurant,Chinese Restaurant,Frozen Yogurt Shop,Lounge,Deli / Bodega
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Mexican Restaurant,Coffee Shop,Café,Deli / Bodega,Yoga Studio,School,Cocktail Bar,Caribbean Restaurant,Indian Restaurant,Pizza Place


In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Discussion
From Clustering, detail common venues by Neighborhood needs to be checked.Through this review, final candidate for restaurant will be selected.

### Cluster 2 (blue cluster) was selected for detail analysis

In [29]:
blue=manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
blue

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Asian Restaurant,Sandwich Place,American Restaurant,Ice Cream Shop,Bar,Noodle House,Vietnamese Restaurant,Salon / Barbershop,Malay Restaurant
6,Central Harlem,African Restaurant,Seafood Restaurant,Fried Chicken Joint,American Restaurant,Chinese Restaurant,French Restaurant,Gym / Fitness Center,Cosmetics Shop,Bookstore,Library
8,Upper East Side,Hotel,Italian Restaurant,American Restaurant,Boutique,Art Gallery,Bakery,Spa,Coffee Shop,Plaza,Sushi Restaurant
10,Lenox Hill,Gym / Fitness Center,Gym,Burger Joint,Coffee Shop,Thai Restaurant,Wine Shop,Bakery,Italian Restaurant,Turkish Restaurant,Gift Shop
12,Upper West Side,Italian Restaurant,American Restaurant,Vegetarian / Vegan Restaurant,Burger Joint,Seafood Restaurant,Bar,Wine Bar,Bakery,Ice Cream Shop,Beer Store
15,Midtown,Bookstore,Park,Clothing Store,Hotel,Sporting Goods Shop,Bakery,Cocktail Bar,French Restaurant,Coffee Shop,Chinese Restaurant
16,Murray Hill,Hotel,Japanese Restaurant,Burger Joint,Italian Restaurant,Coffee Shop,Jewish Restaurant,Bagel Shop,Cocktail Bar,Spa,Liquor Store
17,Chelsea,Ice Cream Shop,Hotel,Nightclub,Seafood Restaurant,Theater,Bakery,Coffee Shop,Asian Restaurant,Italian Restaurant,Speakeasy
18,Greenwich Village,Italian Restaurant,Sushi Restaurant,French Restaurant,Café,Clothing Store,Seafood Restaurant,Gourmet Shop,Cosmetics Shop,Coffee Shop,Optical Shop
20,Lower East Side,Café,Coffee Shop,Latin American Restaurant,Shoe Store,Japanese Restaurant,Ramen Restaurant,Art Gallery,Cocktail Bar,Clothing Store,Bubble Tea Shop


### Excluding Restauranst from 1st Most Common Venue

In [36]:
Blue1=blue[~blue['1st Most Common Venue'].str.contains("Restaurant")]
Blue1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Hotel,Italian Restaurant,American Restaurant,Boutique,Art Gallery,Bakery,Spa,Coffee Shop,Plaza,Sushi Restaurant
10,Lenox Hill,Gym / Fitness Center,Gym,Burger Joint,Coffee Shop,Thai Restaurant,Wine Shop,Bakery,Italian Restaurant,Turkish Restaurant,Gift Shop
15,Midtown,Bookstore,Park,Clothing Store,Hotel,Sporting Goods Shop,Bakery,Cocktail Bar,French Restaurant,Coffee Shop,Chinese Restaurant
16,Murray Hill,Hotel,Japanese Restaurant,Burger Joint,Italian Restaurant,Coffee Shop,Jewish Restaurant,Bagel Shop,Cocktail Bar,Spa,Liquor Store
17,Chelsea,Ice Cream Shop,Hotel,Nightclub,Seafood Restaurant,Theater,Bakery,Coffee Shop,Asian Restaurant,Italian Restaurant,Speakeasy
20,Lower East Side,Café,Coffee Shop,Latin American Restaurant,Shoe Store,Japanese Restaurant,Ramen Restaurant,Art Gallery,Cocktail Bar,Clothing Store,Bubble Tea Shop
23,Soho,Clothing Store,Women's Store,Shoe Store,Men's Store,Boutique,Art Gallery,Salon / Barbershop,Yoga Studio,Candy Store,Cupcake Shop
27,Gramercy,Thrift / Vintage Store,Bagel Shop,Pizza Place,Cocktail Bar,Coffee Shop,Spa,Restaurant,Italian Restaurant,Hotel,Mexican Restaurant
30,Carnegie Hill,Spa,Bookstore,Pizza Place,Yoga Studio,Italian Restaurant,French Restaurant,Coffee Shop,Café,Gym / Fitness Center,Bakery
32,Civic Center,Gym / Fitness Center,Cocktail Bar,Bakery,Italian Restaurant,Spa,Sandwich Place,Coffee Shop,Sushi Restaurant,Falafel Restaurant,Gym


In [39]:
Blue2=Blue1[~Blue1['2nd Most Common Venue'].str.contains("Restaurant")]
Blue2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Lenox Hill,Gym / Fitness Center,Gym,Burger Joint,Coffee Shop,Thai Restaurant,Wine Shop,Bakery,Italian Restaurant,Turkish Restaurant,Gift Shop
15,Midtown,Bookstore,Park,Clothing Store,Hotel,Sporting Goods Shop,Bakery,Cocktail Bar,French Restaurant,Coffee Shop,Chinese Restaurant
17,Chelsea,Ice Cream Shop,Hotel,Nightclub,Seafood Restaurant,Theater,Bakery,Coffee Shop,Asian Restaurant,Italian Restaurant,Speakeasy
20,Lower East Side,Café,Coffee Shop,Latin American Restaurant,Shoe Store,Japanese Restaurant,Ramen Restaurant,Art Gallery,Cocktail Bar,Clothing Store,Bubble Tea Shop
23,Soho,Clothing Store,Women's Store,Shoe Store,Men's Store,Boutique,Art Gallery,Salon / Barbershop,Yoga Studio,Candy Store,Cupcake Shop
27,Gramercy,Thrift / Vintage Store,Bagel Shop,Pizza Place,Cocktail Bar,Coffee Shop,Spa,Restaurant,Italian Restaurant,Hotel,Mexican Restaurant
30,Carnegie Hill,Spa,Bookstore,Pizza Place,Yoga Studio,Italian Restaurant,French Restaurant,Coffee Shop,Café,Gym / Fitness Center,Bakery
32,Civic Center,Gym / Fitness Center,Cocktail Bar,Bakery,Italian Restaurant,Spa,Sandwich Place,Coffee Shop,Sushi Restaurant,Falafel Restaurant,Gym
35,Turtle Bay,Noodle House,Coffee Shop,Italian Restaurant,Greek Restaurant,Wine Bar,American Restaurant,Sushi Restaurant,French Restaurant,Karaoke Bar,Steakhouse
38,Flatiron,Gym / Fitness Center,Cycle Studio,Gym,American Restaurant,Wine Shop,Bakery,Japanese Restaurant,Furniture / Home Store,Mexican Restaurant,Women's Store


In [40]:
Blue3=Blue2[~Blue2['3rd Most Common Venue'].str.contains("Restaurant")]
Blue3

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Lenox Hill,Gym / Fitness Center,Gym,Burger Joint,Coffee Shop,Thai Restaurant,Wine Shop,Bakery,Italian Restaurant,Turkish Restaurant,Gift Shop
15,Midtown,Bookstore,Park,Clothing Store,Hotel,Sporting Goods Shop,Bakery,Cocktail Bar,French Restaurant,Coffee Shop,Chinese Restaurant
17,Chelsea,Ice Cream Shop,Hotel,Nightclub,Seafood Restaurant,Theater,Bakery,Coffee Shop,Asian Restaurant,Italian Restaurant,Speakeasy
23,Soho,Clothing Store,Women's Store,Shoe Store,Men's Store,Boutique,Art Gallery,Salon / Barbershop,Yoga Studio,Candy Store,Cupcake Shop
27,Gramercy,Thrift / Vintage Store,Bagel Shop,Pizza Place,Cocktail Bar,Coffee Shop,Spa,Restaurant,Italian Restaurant,Hotel,Mexican Restaurant
30,Carnegie Hill,Spa,Bookstore,Pizza Place,Yoga Studio,Italian Restaurant,French Restaurant,Coffee Shop,Café,Gym / Fitness Center,Bakery
32,Civic Center,Gym / Fitness Center,Cocktail Bar,Bakery,Italian Restaurant,Spa,Sandwich Place,Coffee Shop,Sushi Restaurant,Falafel Restaurant,Gym
38,Flatiron,Gym / Fitness Center,Cycle Studio,Gym,American Restaurant,Wine Shop,Bakery,Japanese Restaurant,Furniture / Home Store,Mexican Restaurant,Women's Store


### Pizza was also excluded from restaurant candidate

In [41]:
Blue4=Blue3[~Blue3['3rd Most Common Venue'].str.contains("Pizza")]
Blue4

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Lenox Hill,Gym / Fitness Center,Gym,Burger Joint,Coffee Shop,Thai Restaurant,Wine Shop,Bakery,Italian Restaurant,Turkish Restaurant,Gift Shop
15,Midtown,Bookstore,Park,Clothing Store,Hotel,Sporting Goods Shop,Bakery,Cocktail Bar,French Restaurant,Coffee Shop,Chinese Restaurant
17,Chelsea,Ice Cream Shop,Hotel,Nightclub,Seafood Restaurant,Theater,Bakery,Coffee Shop,Asian Restaurant,Italian Restaurant,Speakeasy
23,Soho,Clothing Store,Women's Store,Shoe Store,Men's Store,Boutique,Art Gallery,Salon / Barbershop,Yoga Studio,Candy Store,Cupcake Shop
32,Civic Center,Gym / Fitness Center,Cocktail Bar,Bakery,Italian Restaurant,Spa,Sandwich Place,Coffee Shop,Sushi Restaurant,Falafel Restaurant,Gym
38,Flatiron,Gym / Fitness Center,Cycle Studio,Gym,American Restaurant,Wine Shop,Bakery,Japanese Restaurant,Furniture / Home Store,Mexican Restaurant,Women's Store


# Result

### From 6 candidates, Lenox Hill, Civic Center and Flatiron were excluded. It is not recommended as a restaurant candidate where Gym/Fitness Center is the "1st Most Common Venue". 

In [42]:
Blue5=Blue4[~Blue4['1st Most Common Venue'].str.contains("Gym")]
Blue5

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Midtown,Bookstore,Park,Clothing Store,Hotel,Sporting Goods Shop,Bakery,Cocktail Bar,French Restaurant,Coffee Shop,Chinese Restaurant
17,Chelsea,Ice Cream Shop,Hotel,Nightclub,Seafood Restaurant,Theater,Bakery,Coffee Shop,Asian Restaurant,Italian Restaurant,Speakeasy
23,Soho,Clothing Store,Women's Store,Shoe Store,Men's Store,Boutique,Art Gallery,Salon / Barbershop,Yoga Studio,Candy Store,Cupcake Shop


### 3 neighborhoods are recommended because it is not red ocean(there is no restaurant in 1st~3rd common venues).  

# Conclusion

## My recommendation is "Soho" for new restaurant opening. In Soho, there are many clothing store and women's store. It means that many women comes to Soho. It can be benefit for new restaurant. Especially, Italian restaurant is recommended in line with the preference of many women. 